In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import plotly.express as px
from datetime import date, datetime, time, timedelta
import numpy as np
token = 'pk.eyJ1IjoidmFsZW50aW5hbGF2ZXJkZSIsImEiOiJja3lkODV6dnAwMzNtMnVxaGx1ZjNobjUzIn0.q98o5a_hvqz2O4RI3DNmTw'# MAPBOX

# Lectura de datos

In [ ]:
data= {}
#Se lee la matriz de distancia y se orgamiza de forma que la primera fila y columna sea la casa del rep
matriz_distancia=pd.read_csv('matriz_distancia_chris.csv').set_index('id')
matriz_distancia=matriz_distancia.loc[matriz_distancia.index[::-1],matriz_distancia.columns[::-1]]
a=matriz_distancia.isna().sum(axis=0)
zz=a[a!=0].index
matriz_distancia=matriz_distancia.loc[~matriz_distancia.index.isin(zz.astype(int)),set(matriz_distancia.columns)-set(zz)]


matriz_tiempo=pd.read_csv('matriz_tiempo_chris.csv').set_index('id')
matriz_tiempo=matriz_tiempo.loc[matriz_tiempo.index[::-1],matriz_tiempo.columns[::-1]]

b=matriz_tiempo.isna().sum(axis=0)
xx=a[a!=0].index

#A la matriz de tiempo se le añade 7 minutos que se tardaría en cada cuenta
matriz_tiempo=matriz_tiempo.loc[~matriz_tiempo.index.isin(zz.astype(int)),set(matriz_tiempo.columns)-set(zz)]
matriz_tiempo=matriz_tiempo+7

#Se leen las probabilidades de las cuentas
clientes=pd.read_excel('accountsProbabilitiesChris.xlsx')
clientes['id_real']=clientes['id'].astype(int).astype(str)+'_'+clientes['Practice Name']
clientes['Buy Probability']=clientes['Buy Probability'].replace('%','',regex=True).astype(float)/100
clientes['id']=clientes['id'].astype(int)
clientes.set_index('id',inplace=True)
clientes=clientes[clientes['Buy Probability']>=0]
clientes=clientes.loc[clientes.index.isin(matriz_distancia.index)]

matriz_tiempo=matriz_tiempo.loc[clientes.index,clientes.index.astype(str)]
matriz_distancia=matriz_distancia.loc[clientes.index,clientes.index.astype(str)]

clientes['prospect']=pd.isna(clientes['Last Buy Date']).astype(int)

#La matriz de costo va a ser igual a la matriz distancia * (1 - probabilidad)
for i in matriz_distancia.index:
    try:
        matriz_distancia[str(i)]=matriz_distancia[str(i)]*(1-clientes.loc[i,'Buy Probability'])
    except:
        pass

data['time_matrix'] = matriz_tiempo.astype(int).values
data['distancia'] =  matriz_distancia.astype(int).values



clientes=clientes.loc[matriz_tiempo.index]
clientes=clientes.reset_index()

#El número de vehiculos es igual al número de dias totales para recorrer las cuentas 
data['num_vehicles'] = 14
data['depot'] = 0

# Creación del modelo

In [ ]:
manager = pywrapcp.RoutingIndexManager(len(data['distancia']),data['num_vehicles'],data['depot'])
routing = pywrapcp.RoutingModel(manager)
# Se asigna como pesos del grafo la matriz de distancia
def weight_callback(from_index, to_index):
    """Returns the weight between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distancia'][from_node][to_node]



transit_callback_index = routing.RegisterTransitCallback(weight_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Se añade la matriz de tiempo al modelo
def time_callback(from_index, to_index):
    """Returns the time between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node] 

transit_callback_index_0 = routing.RegisterTransitCallback(time_callback)

#Se añade la restricción de tiempo al modelo para que la ruta no pase los 420 minutos
dim = 'Tiempo'
routing.AddDimension(transit_callback_index_0,
        0,  # no slack
        420,  # vehicle maximum travel time
        True,  # start cumul to zero
        dim)
time_dimension = routing.GetDimensionOrDie(dim)
time_dimension.SetGlobalSpanCostCoefficient(100)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

#Se resuelve la ruta
solution = routing.SolveWithParameters(search_parameters)


# Impresión de solución

In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    #print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    max_=0
    df=pd.DataFrame()

    #Los vehiculos en este caso son tomados como los dias necesarios para recorrer todas las cuentas
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for day {}:\n'.format(vehicle_id+1)
        route_distance = 0
        clientes_=0
        probabilidad=0
        #Se crea dataframe con la información de rutas de cada dia
        data=pd.DataFrame(columns=['Day','Id','Practice Name','Buy Probability','City','Last Buy Date','Last Visit Date','LV Outcome','accountstatus','accountrating'])
        while not routing.IsEnd(index):
            clientes_+=1
            plan_output += ' {} -> '.format(clientes.loc[manager.IndexToNode(index),'Practice Name'])
            probabilidad += clientes.loc[manager.IndexToNode(index),'Buy Probability']
            
            df2 = {
                'Day':vehicle_id,
                'Id':clientes.loc[manager.IndexToNode(index),'id'],
                'Practice Name': clientes.loc[manager.IndexToNode(index),'Practice Name'],
                'Buy Probability': clientes.loc[manager.IndexToNode(index),'Buy Probability'],
                'City': clientes.loc[manager.IndexToNode(index),'city'],
                'Last Buy Date':clientes.loc[manager.IndexToNode(index),'Last Buy Date'],
                'Last Visit Date':clientes.loc[manager.IndexToNode(index),'Last Visit Date'],
                'LV Outcome':clientes.loc[manager.IndexToNode(index),'LV Outcome'],
                'accountstatus':clientes.loc[manager.IndexToNode(index),'accountstatus'],
                'accountrating':clientes.loc[manager.IndexToNode(index),'accountrating'],
                }
            data = data.append(df2, ignore_index = True)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(clientes.loc[manager.IndexToNode(index),'Practice Name'])
        plan_output += 'Distance of the route: {} m'.format(route_distance)
        max_route_distance = max(route_distance, max_route_distance)
        df=df.append(data,ignore_index=True)
    return df

In [ ]:
resultado=print_solution(data, manager, routing, solution)